In [1]:
import pandas as pd

In [2]:
train_data = pd.read_parquet('./fhv_tripdata_2021-01.parquet')
test_data = pd.read_parquet('./fhv_tripdata_2021-02.parquet')
train_data['duration']=(train_data.dropOff_datetime-train_data.pickup_datetime)
train_data['duration'] = train_data.duration.dt.total_seconds()/60

In [3]:
from sklearn.feature_extraction import DictVectorizer

In [4]:
# train_data.head(5)

In [5]:
# train_data.duration.plot()

In [6]:
train_data.duration.quantile(0.95), train_data.duration.quantile(0.05)

(47.25, 3.0166666666666666)

In [7]:
train_data[(train_data.duration > 48)].shape

(55245, 8)

In [8]:
train_data[(train_data.duration < 3)].shape

(55959, 8)

In [ ]:
#let's use small rides that are under an hour

In [9]:
train_data = train_data[(train_data.duration <= 60) & (train_data.duration >= 1)]

In [10]:
train_data.shape

(1109826, 8)

In [11]:
train_data.duration.median()

13.233333333333333

In [12]:
train_data

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [13]:
train_data.PUlocationID.fillna(-1,inplace=True)
train_data.DOlocationID.fillna(-1,inplace=True)

In [14]:
train_data.PUlocationID.isnull().sum()

0

In [15]:
train_data.PUlocationID.notnull().sum()

1109826

In [16]:
train_data.PUlocationID.isnull().sum() + train_data.PUlocationID.notnull().sum(), train_data.shape[0]

(1109826, 1109826)

In [17]:
train_data.PUlocationID.isnull().sum(),train_data.DOlocationID.isnull().sum()

(0, 0)

In [18]:
train_data.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [19]:
train_data.PUlocationID = train_data.PUlocationID.astype('int').astype('str')
train_data.DOlocationID = train_data.DOlocationID.astype('int').astype('str') #float to int and then to str

In [20]:
train_data.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                      object
DOlocationID                      object
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [21]:
train_data

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1,-1,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1,-1,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1,72,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1,61,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1,71,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7,7,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44,91,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171,171,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15,15,None,B03285,19.433333


In [22]:
train_data.PUlocationID.value_counts()[-1], train_data.DOlocationID.value_counts()[-1]

(1, 1)

In [23]:
categorical_cols = ['PUlocationID','DOlocationID']
# numerical_cols = ['duration']

train_data.categorical_cols = train_data[categorical_cols]

/usr/local/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  after removing the cwd from sys.path.


In [24]:
training_dict = train_data[categorical_cols].to_dict(orient='records')
# training_dict

In [25]:
len(training_dict)

1109826

In [26]:
dv = DictVectorizer()
X_train = dv.fit_transform(training_dict)

target = 'duration'
y_train = train_data[target].values

In [27]:
X_train.shape

(1109826, 525)

In [28]:
X_train

<1109826x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2219652 stored elements in Compressed Sparse Row format>

In [29]:
# dv.feature_names_

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [31]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

10.528519107211451

In [32]:
# help(lr)

In [33]:
test_data['duration']=(test_data.dropOff_datetime-test_data.pickup_datetime)
test_data['duration'] = test_data.duration.dt.total_seconds()/60

In [34]:
test_data = test_data[(test_data.duration <= 60) & (test_data.duration >= 1)]

In [35]:
test_data.PUlocationID.fillna(-1,inplace=True)
test_data.DOlocationID.fillna(-1,inplace=True)
# test_data.PUlocationID = test_data.PUlocationID.astype('int').astype('str')
# test_data.DOlocationID = test_data.DOlocationID.astype('int').astype('str')

In [36]:
test_data.PUlocationID = test_data.PUlocationID.astype('int').astype('str')
test_data.DOlocationID = test_data.DOlocationID.astype('int').astype('str')

In [37]:
testing_dict = test_data[['PUlocationID','DOlocationID']].to_dict(orient='records')

In [38]:
len(testing_dict)

990113

In [39]:
X_test = dv.transform(testing_dict)
X_test.shape

(990113, 525)

In [40]:
y_test = test_data.duration.values

In [41]:
y_test_pred = lr.predict(X_test)

In [42]:
mean_squared_error(y_test,y_test_pred,squared=False)

11.014283196016624

In [44]:
train_data.shape

(1109826, 8)

In [55]:
train_data.iloc[:5,:1]

,dispatching_base_num
0,B00009
1,B00009
3,B00037
4,B00037
5,B00037
